In [ ]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
!pip install pytorch 
import matplotlib.pyplot as plt

In [ ]:
"""
#connect with drive
from google.colab import drive
drive.mount('/content/drive')
"""

In [ ]:
"""
# Path to data:
# David's path
path = '/content/drive/MyDrive/Dengue_GIS_Visualization/DengueData/'
# Dana's path
#path='/content/drive/MyDrive/Dengue_GIS Visualization/Dengue_GIS_Visualization/DengueData/'
"""

In [ ]:
# Read Data
merge_cases_temp_precip = pd.read_csv('Data/merge_cases_temperature_WeeklyPrecipitation_timeseries.csv')
# Remove extra column
merge_cases_temp_precip = merge_cases_temp_precip.drop('Unnamed: 0', 1)
merge_cases_temp_precip.LastDayWeek = pd.to_datetime(merge_cases_temp_precip.LastDayWeek)
merge_cases_temp_precip

# Time Series

## Data visualization

In [ ]:
# Dengue cases in time
def timeseries (x_axis, y_axis, x_label):
    plt.figure(figsize = (12, 8))
    plt.plot(x_axis, y_axis, color ='black')
    plt.xlabel(x_label) 
    plt.ylabel('Dengue Cases')

timeseries(merge_cases_temp_precip['LastDayWeek'], merge_cases_temp_precip['cases_medellin'], 'cases by Week')

## DataSet

In [ ]:
dataset = merge_cases_temp_precip[['temperature_medellin','percipitation_medellin','cases_medellin']]
dataset.index = merge_cases_temp_precip.LastDayWeek
dataset  #DF

# Prepare data to supervised learning time series

we will use:
* data: is the dataframe in our case (Dengue Cases, Precipitation and Temperature)
* n_in: is the number of lag weeks in the past (length of window)

The heart of this "series_to_supervised" function is the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html">shift</a> fuction of pandas

This function gets as input the number of periods(in this case the number of weeks represented as rows up or down in the dataframe) to move the columns of a dataframe.
E.g. 
* If we have merge_cases_temp_precip['cases_medellin'].shift(1) all the rows of column cases_medellin will move one row down
* If we have merge_cases_temp_precip['cases_medellin'].shift(-1) all the rows of column cases_medellin will move one row up


In [ ]:
# prepare data for lstm
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 

### normalize features
As we are working with a Neural Network the data values ​​must be normalized to help backpropagation algorithm
So we will use the <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html">MinMaxScaler</a> from sklearn

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1)) # Scaler between 0 and 1
scaled = scaler.fit_transform(dataset) # As we can see data set has 3 Columns (This shape is also important for inverse scaler as we will see in future)

In [ ]:
# length of window
weeks = 10

# frame as supervised learning
data = series_to_supervised(scaled, n_in=weeks)
DataFrame(data).head()

## Features Set

In [ ]:
# We define the number of features as 3 (Temperature, Precipitation and Dengue Cases)
n_features = 3
# The features to train the model will be all except the values of the actual week 
# We can't use the temperature and precipitation in week t because whe need to resample a a 3D Array
features_set = DataFrame(data.values[:,:-n_features])
# Convert pandas data frame to np.array to reshape as 3D Array
features_set = features_set.to_numpy()
features_set

## Labels Set

In [ ]:
# We will use Dengue cases in last week 
labels_set = DataFrame(data.values[:,-1])
# Convert pandas data frame to np.array
labels_set = labels_set.to_numpy()
labels_set

## Train Test Split

In [ ]:
# We need a sequence so we can't split randomly
# To divide into Train (90%) and test (10%) to do that we need to know the 90% of the total dataframe
size = features_set.shape[0]
split = int(size*(9/10))

### train

In [ ]:
# We will train with 1st 90% of data and test with last 10%
train_X = features_set[:split] ##90% train
train_y = labels_set[:split]  ##90% train

### test

In [ ]:
test_X = features_set[split:] ##10% test
test_y = labels_set[split:] ##10% test

## Reshape

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], weeks, n_features))
test_X = test_X.reshape((test_X.shape[0], weeks, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

### Transform ndarray to tensor pytorch

In [ ]:
import torch
train_X = torch.from_numpy(train_X)
train_y = torch.from_numpy(train_y)
test_X = torch.from_numpy(test_X)
test_y = torch.from_numpy(test_y)

## Model

### LSTM

In [ ]:
""" TODO: Pytorch Model implementation"""
import torch.nn as nn
from torch.autograd import Variable

import random
random.seed(0)

torch.manual_seed(0)


# num_classes: weeks to predict (1)
# num_layers: # of activations from past LSTM layers
# input_size: num of features (n_features = 3)
# hidden_size: # Neurons in hidden layer (50)
# seq_length: length of window (weeks)

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # Define LSTM layer with hidden_size neurons, input_size inputs and 1 ho and c0
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        # Define ouput linear layer takes hidden_size inputs, and num_classes outputs
        self.fc = nn.Linear(hidden_size, num_classes)
        

    def forward(self, x):
        
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        # print(f'last activation: {h_out}')
        # print(f'output: {ula[:,-1,:]}') # Last Activation is output in last position
        #print(h_out.shape)
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)
        
        return out

In [ ]:
# Create instance of nn
num_epochs = 2000
learning_rate = 0.01

input_size = n_features # Features
hidden_size = 60 # LSTM layer neurons
num_layers = 1 # Number of LSTM layers
num_classes = 1 # Output Neurons

# Instance
lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
lstm = lstm.float()

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)


# fit network
# Train the model
for epoch in range(num_epochs):
    outputs = lstm(train_X.float())
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, train_y.float())
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

# Test

In [ ]:
from math import sqrt
from numpy import concatenate

# make a prediction
lstm.eval()
train_predict = lstm(test_X.float())
yhat = train_predict.detach().numpy()

In [ ]:
yhat.shape

In [ ]:
# Convert test data to 2D 
test_X = test_X.reshape((test_X.shape[0], weeks*n_features))

# invert scaling for forecast
# As we said Scaler needs 3 columns so we can take those columns from test data and take again the predictions
# Concatenate last 2 columns of test data with predicted data (yhat)
inv_yhat = concatenate((test_X[:, -(n_features-1):], yhat), axis=1)
# Inverse Scaler
inv_yhat = scaler.inverse_transform(inv_yhat)
# Take predicted data scaled to original Dengue cases
inv_yhat = inv_yhat[:,-1]

# invert scaling for actual
# Same process than for predicted data (yhat)
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_X[:, -(n_features-1):], test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,-1]

In [ ]:
from sklearn.metrics import  mean_absolute_error

# calculate MAE
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

#### Plot predicted vs actual dengue cases

In [ ]:
data_predict = inv_yhat  ## predicted target  cases
dataY_plot = inv_y  ##  real test-target cases

data_predict = data_predict.reshape(len(data_predict), 1)
dataY_plot = dataY_plot.reshape(len(dataY_plot), 1)

import matplotlib.pyplot as plt

plt.plot(dataY_plot, label = 'actual')
plt.plot(data_predict, label = 'predicted')
plt.legend(loc="upper left")

plt.suptitle('Time-Series Prediction')
plt.show()

<ol>
  <li> <a href="https://towardsdatascience.com/predictive-analytics-time-series-forecasting-with-gru-and-bilstm-in-tensorflow-87588c852915">Predictive Analytics: Time-Series Forecasting with GRU and BiLSTM in TensorFlow</a></li>
  <li><a href="https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/">Multivariate Time Series Forecasting with LSTMs in Keras</a></li>
</ol>